In [1]:
import plotly.express as px
import cip
import pandas as pd
from cip_interpreter import ForwardRateInterpreter
import textwrap

In [2]:
# Input Variables
domestic_rate_annual = 0.06
foreign_rate_annual = 0.02

spot_t = 1.32

tenors_days = [7, 30, 60, 90, 180, 270, 365]

In [3]:
# Get forward rates by tenor in a dataframe
forward_rates = cip.calculate_multiple_forward_rates(domestic_rate_annual, foreign_rate_annual, tenors_days, spot_t)

forward_rates_df = pd.DataFrame({
    "Tenor (Days)": tenors_days, 
    "Forward Rate": forward_rates
})

In [4]:
forward_rates_df

,Tenor (Days),Forward Rate
0,7,1.321026
1,30,1.324393
2,60,1.328771
3,90,1.333134
4,180,1.346139
5,270,1.359015
6,365,1.372469


In [ ]:
# Plot forward curve
tenor_labels = ["1W", "1M", "2M", "3M", "6M", "9M", "1Y"]

fig = px.line(
    forward_rates_df,
    x="Tenor (Days)",
    y="Forward Rate",
    title="FX Forward Curve (Simplified)",
    markers=True
)
fig.update_xaxes(tickvals=tenors_days, ticktext=tenor_labels, title_text="Maturity")
fig.update_yaxes(title_text="Forward Rate")
fig.add_hline(
    y=spot_t,
    line_dash="dash",
    line_width=2,
    line_color="red",
    annotation_text=f"Spot ({spot_t:.4f})",
    annotation_position="bottom left",
)
fig.show()

forward_rate_value = float(forward_rates_df.iloc[0]["Forward Rate"])
interpreter = ForwardRateInterpreter(
    forward_rate=forward_rate_value,
    spot_rate=spot_t,
    tenor_days=tenors_days[0]
)

for i, s in enumerate(interpreter.get_detailed_interpretations(), start=1):
    print(f"{i}. {textwrap.fill(s, width=90, subsequent_indent='   ')}\n")

1. The forward rate is greater than the spot rate which implies that domestic rates are
   higher than foreign rates.

2. Intuition: If the domestic rates are higher than foreign rates then the forward needs to
   be higher (foreign currency appreciation) than spot to makeup for higher domestic
   payoff.

3. 1 unit of foreign currency will buy more domestic currency at time T (today) than it does
   at today's spot (foreign currency is at a forward premium)

4. Also means the same amount of domestic currency will buy less foreign currency at time T
   today than it does today's spot (domestic currency is at a forward discount)

5. Domestic currency forward must be 0.08% cheaper (relative to spot) to prevent arbitrage.

6. Foreign currency forward must be 0.08% more expensive (relative to spot) to prevent
   arbitrage.

